## Tensorflow Keras for NLP

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import optimizers
from keras import losses
from keras import activations
from keras import callbacks
from keras import utils
from keras import metrics
from keras import applications
from keras import regularizers
from keras import preprocessing

2025-02-09 19:56:25.655460: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739105785.786637    5161 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739105785.828297    5161 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-09 19:56:26.133565: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
!nvidia-smi -L

/usr/bin/zsh: /home/lovecrush/miniconda3/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
GPU 0: NVIDIA GeForce GTX 1650 (UUID: GPU-72ac3021-d1f7-0394-e845-34821fc5a1c7)


In [3]:
# Download helper functions script
# !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [4]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [5]:
# # Download data (same as from Kaggle)
# !wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# # Unzip data
# unzip_data("nlp_getting_started.zip")



In [6]:

# Delete the zip file
# !rm "nlp_getting_started.zip"


### *Visualizing a Text Dataset*

In [7]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [8]:
# Shuffle the training data
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled


,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [9]:
train_df["target"].value_counts().to_frame()

,count
target,
0,4342
1,3271


In [10]:
# Let visualize some random training examples
import random

random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")


Target: 0 (not real disaster)
Text:
Kids got Disney version of the game Operation only 2 AA batteries? I swear my old version had like 8 Ds and would nearly electrocute you.

---

Target: 1 (real disaster)
Text:
@antpips67 @JohnEJefferson obviously I'm aware that not all AS are from countries we have bombed but a lot are fleeing conflict

---

Target: 0 (not real disaster)
Text:
the way he did me  destroyed me...

---

Target: 0 (not real disaster)
Text:
Niagara Falls Package Deals Deluge Tours YYESO

---

Target: 0 (not real disaster)
Text:
@Casper_rmg u on dick

---



In [11]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of training data for validation split
                                                                            random_state=42)


In [12]:
# Let's check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [13]:
# View the first 10 training sentences and their labels
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

### *Converting Text into a number*

In [14]:
text_vectorizer = layers.TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <OOV>)
                                            standardize="lower_and_strip_punctuation", # how to process text
                                            split="whitespace", # how to split tokens
                                            ngrams=None, # create groups of n-words?
                                            output_mode="int", # how to map tokens to numbers
                                            output_sequence_length=None, # how long should the output sequence of tokens be
                                            # pad_to_max_tokens=True
                                            )



I0000 00:00:1739105804.060610    5161 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2795 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


In [15]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))


15

In [16]:
# Setup text vectorization with custom variables
max_vocab_length = 10000            # max number of words to have in our vocabulary
max_length = 15                     # max length our sequences will be (e.g. how many words from a tweet does a model see?)

text_vectorizer = layers.TextVectorization(max_tokens=max_vocab_length,
                                            output_mode="int",
                                            output_sequence_length=max_length)


In [17]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [18]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # most common words (note: the first word is <OOV>)
bottom_5_words = words_in_vocab[-5:] # least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")


Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]
Bottom 5 least common words: [np.str_('pages'), np.str_('paeds'), np.str_('pads'), np.str_('padres'), np.str_('paddytomlinson1')]


### *Word Embedding*

In [20]:
tf.random.set_seed(42)

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                            output_dim=128, # set size of embedding vector
                            embeddings_initializer="uniform", # default, intialize randomly
                            input_length=max_length,
                            name="embedding_1") # how long is each input

embedding



/home/lovecrush/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


<Embedding name=embedding_1, built=False>

In [21]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:", random_sentence, '\n')

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
print(f"Embedded tensor: ", '\n',sample_embed)

Original text: @asukager magical bag of blazing 

Embedded tensor:  
 tf.Tensor(
[[[ 0.02026704 -0.02446166  0.02934915 ... -0.02575492  0.0282092
   -0.03685801]
  [ 0.02026704 -0.02446166  0.02934915 ... -0.02575492  0.0282092
   -0.03685801]
  [-0.02177768 -0.01254987 -0.02649635 ...  0.02922404  0.00263631
   -0.03111788]
  ...
  [ 0.01211643  0.02169497  0.02045457 ...  0.00584284  0.01382774
   -0.0222173 ]
  [ 0.01211643  0.02169497  0.02045457 ...  0.00584284  0.01382774
   -0.0222173 ]
  [ 0.01211643  0.02169497  0.02045457 ...  0.00584284  0.01382774
   -0.0222173 ]]], shape=(1, 15, 128), dtype=float32)


In [22]:
sample_embed.shape

TensorShape([1, 15, 128])

In [23]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 2.02670358e-02, -2.44616624e-02,  2.93491520e-02, -2.52246391e-02,
       -4.77392077e-02, -2.75766011e-02,  1.79052614e-02, -1.13523006e-03,
        9.97120142e-03, -3.55717428e-02, -2.31703874e-02, -4.78212610e-02,
       -2.83162724e-02,  4.97269370e-02, -1.79532394e-02, -1.32881775e-02,
        4.36810963e-02, -4.53053787e-03,  2.83894576e-02, -2.16117259e-02,
        1.24958977e-02,  1.14750117e-04,  3.65015604e-02,  4.32580821e-02,
       -4.98794205e-02, -2.50036474e-02, -2.24821270e-05,  3.04201730e-02,
       -3.09182331e-03,  1.73325874e-02, -1.62220225e-02, -4.11677733e-02,
       -3.94229293e-02,  4.61145155e-02, -5.85144758e-03, -2.34682914e-02,
       -1.45771876e-02, -8.74332339e-03, -4.28928733e-02,  2.80493610e-02,
       -2.87862550e-02, -8.25233385e-03,  2.01166756e-02, -1.00207217e-02,
        4.66999076e-02,  2.44720094e-02,  2.19640769e-02, -1.51390806e-02,
        2.72255018e-03, -1.88796874e-02,  3.31459977

### *Model a Multinomial Naive Bayes*

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Create a tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieve the accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieve the accuracy of: 79.27%


In [26]:
# Make some predictions
baseline_pred = model_0.predict(val_sentences)
baseline_pred[:5]

array([1, 1, 1, 0, 0])

In [27]:
val_labels

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In [28]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [29]:
# get baseline results
baseline_results = calculate_results(y_true=val_labels, y_pred=baseline_pred)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### *A simple feed-forward model*

In [30]:
input = layers.Input(shape=(1,), dtype=tf.string)        # inputs are 1-dimensional strings
x = text_vectorizer(input)                               # turn the input text into numbers
x = embedding(x)                                         # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x)                   # condense the feature vector for each token to one vector
output = layers.Dense(1, activation="sigmoid")(x)        # create the output layer, want binary output so use sigmoid activation
model_1 = tf.keras.Model(input, output)                  # construct the model

# Compile the model
model_1.compile(loss=losses.binary_crossentropy,
                optimizer=optimizers.Adam(learning_rate=0.01),
                metrics=[metrics.binary_accuracy])

model_1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
# Fit the model
model_1_history = model_1.fit(train_sentences, train_labels,
                            epochs=5,
                            validation_data=(val_sentences, val_labels),
                            callbacks=[create_tensorboard_callback("logs/model_1", "model_1_training")])


Saving TensorBoard log files to: logs/model_1/model_1_training/20250209-195652
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step - binary_accuracy: 0.7325 - loss: 0.5440 - val_binary_accuracy: 0.7900 - val_loss: 0.4648
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - binary_accuracy: 0.8747 - loss: 0.3036 - val_binary_accuracy: 0.7585 - val_loss: 0.5176
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - binary_accuracy: 0.9118 - loss: 0.2224 - val_binary_accuracy: 0.7585 - val_loss: 0.5949
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - binary_accuracy: 0.9359 - loss: 0.1642 - val_binary_accuracy: 0.7651 - val_loss: 0.6768
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - binary_accuracy: 0.9509 - loss: 0.1409 - val_binary_accuracy: 0.7546 - val_loss: 0.7677


In [32]:
model_1.evaluate(val_sentences, val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - binary_accuracy: 0.7244 - loss: 0.8423


[0.7677381634712219, 0.7545931935310364]

In [33]:
embedding.weights

[<Variable path=embedding_1/embeddings, shape=(10000, 128), dtype=float32, value=[[-0.00801276  0.0493915   0.04118228 ... -0.01559352 -0.02304153
   -0.02042138]
  [-0.0223062  -0.05348571 -0.01859231 ... -0.01897147  0.0133254
   -0.01497234]
  [-0.17552383  0.0438375   0.0914645  ... -0.09106003 -0.1232762
   -0.07796458]
  ...
  [ 0.02784327 -0.03139472  0.03411582 ...  0.0266456  -0.01082576
   -0.02701615]
  [-0.12876661  0.06636021  0.11866405 ... -0.12250449 -0.1070995
   -0.08177507]
  [-0.2107999   0.15630655  0.22090071 ... -0.23299113 -0.13467468
   -0.2080563 ]]>]

In [34]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape) # same size as vocab size and embedding_dim (each word is a embedding_dim size vector)

(10000, 128)


In [35]:
model_1_pred_probs = model_1.predict(val_sentences)
print(f"{model_1_pred_probs.shape}")

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step
(762, 1)


In [36]:
model_1_pred = tf.squeeze(tf.round(model_1_pred_probs))
model_1_pred

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0.,
       1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [37]:
model_1_results = calculate_results(y_true=val_labels, y_pred=model_1_pred)
model_1_results

{'accuracy': 75.45931758530183,
 'precision': 0.7542299722656125,
 'recall': 0.7545931758530183,
 'f1': 0.7537160541571394}

### *Visualizing learned embeddings*

In [38]:
# get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000,
 ['',
  '[UNK]',
  np.str_('the'),
  np.str_('a'),
  np.str_('in'),
  np.str_('to'),
  np.str_('of'),
  np.str_('and'),
  np.str_('i'),
  np.str_('is')])

In [39]:
model_1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,840,389 (14.65 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,560,260 (9.77 MB)

In [40]:
# Get the weight matrix of embedding layer
# (There are the numerical patterns between the text in the training dataset the model has learned)
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
embed_weights

array([[-0.00801276,  0.0493915 ,  0.04118228, ..., -0.01559352,
        -0.02304153, -0.02042138],
       [-0.0223062 , -0.05348571, -0.01859231, ..., -0.01897147,
         0.0133254 , -0.01497234],
       [-0.17552383,  0.0438375 ,  0.0914645 , ..., -0.09106003,
        -0.1232762 , -0.07796458],
       ...,
       [ 0.02784327, -0.03139472,  0.03411582, ...,  0.0266456 ,
        -0.01082576, -0.02701615],
       [-0.12876661,  0.06636021,  0.11866405, ..., -0.12250449,
        -0.1070995 , -0.08177507],
       [-0.2107999 ,  0.15630655,  0.22090071, ..., -0.23299113,
        -0.13467468, -0.2080563 ]], dtype=float32)

In [41]:
embed_weights.shape         # same size as vocab size and embedding_dim (each word is a embedding_dim size vector)


(10000, 128)

### *Long Short-Term Memory cell*

In [42]:
# Set a random seed and create embedding layer

tf.random.set_seed(42)

model_2_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")

# Create  LSTM model
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)
print(x.shape)

x = layers.LSTM(64)(x)          # return vector for whole sequence
print(x.shape)

outputs = layers.Dense(1, activation=activations.sigmoid)(x)
model_2 = keras.Model(inputs, outputs, name="model_2_LSTM")

/home/lovecrush/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


(None, 15, 128)
(None, 64)


In [43]:
# Compile the model
model_2.compile(loss=losses.binary_crossentropy,
                optimizer=optimizers.Adam(),
                metrics=[metrics.binary_accuracy])

model_2.summary()

Model: "model_2_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_2 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,329,473 (5.07 MB)

 Trainable params: 1,329,473 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

In [44]:
# Fit model
model_2_history = model_2.fit(train_sentences,train_labels,
                              validation_data=(val_sentences, val_labels),
                              epochs=5,
                              callbacks=[create_tensorboard_callback("logs/model_2_training", "LSTM")])

Saving TensorBoard log files to: logs/model_2_training/LSTM/20250209-195716
Epoch 1/5
  1/215 ━━━━━━━━━━━━━━━━━━━━ 26:58 8s/step - binary_accuracy: 0.5000 - loss: 0.6922

I0000 00:00:1739105844.430167    6163 cuda_dnn.cc:529] Loaded cuDNN version 90300


215/215 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - binary_accuracy: 0.6690 - loss: 0.5875 - val_binary_accuracy: 0.7743 - val_loss: 0.4577
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - binary_accuracy: 0.8661 - loss: 0.3326 - val_binary_accuracy: 0.7651 - val_loss: 0.5027
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - binary_accuracy: 0.9164 - loss: 0.2286 - val_binary_accuracy: 0.7559 - val_loss: 0.6346
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - binary_accuracy: 0.9456 - loss: 0.1532 - val_binary_accuracy: 0.7572 - val_loss: 0.7170
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - binary_accuracy: 0.9648 - loss: 0.1069 - val_binary_accuracy: 0.7651 - val_loss: 0.7171


In [45]:
# make the predictions on the validation dataset
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step


((762, 1),
 array([[0.02205615],
        [0.9334641 ],
        [0.99745196],
        [0.07130566],
        [0.00465293],
        [0.9982838 ],
        [0.34514073],
        [0.99964607],
        [0.99686426],
        [0.35052133]], dtype=float32))

In [46]:
# Round out predictions and reduce to 1-dimensional array
model_2_pred = tf.squeeze(tf.round(model_2_pred_probs))
model_2_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 0.], dtype=float32)>

In [47]:
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_pred)
model_2_results

{'accuracy': 76.50918635170603,
 'precision': 0.7666862986020204,
 'recall': 0.7650918635170604,
 'f1': 0.7628949062414964}

### *GRU - Gated Recurrent Unit*

In [48]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(16)

model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")

# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")

x = text_vectorizer(inputs)
x = model_3_embedding(x)

x = layers.GRU(64, return_sequences=True)(x)   # stacking recurrent cells requires return_sequences=True
x = layers.GRU(64)(x)

x = layers.Dense(64, activation=activations.relu)(x)
outputs = layers.Dense(1, activation=activations.sigmoid)(x)

model_3 = keras.Model(inputs, outputs, name="model_3_gru")


/home/lovecrush/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [49]:
# Compile the GRU model
model_3.compile(loss=losses.binary_crossentropy,
                optimizer=optimizers.Adam(),
                metrics=[metrics.binary_accuracy])

model_3.summary()

Model: "model_3_gru"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_3 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 15, 64)         │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 64)             │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,346,433 (5.14 MB)

 Trainable params: 1,346,433 (5.14 MB)

 Non-trainable params: 0 (0.00 B)

In [50]:
# Fit the model
model_3_history = model_3.fit(train_sentences, train_labels,
                              validation_data=(val_sentences, val_labels),
                              epochs=5,
                              callbacks=[create_tensorboard_callback("logs/model_3_training", "gru")])

Saving TensorBoard log files to: logs/model_3_training/gru/20250209-195755
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 19s 28ms/step - binary_accuracy: 0.6586 - loss: 0.6065 - val_binary_accuracy: 0.7717 - val_loss: 0.4642
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - binary_accuracy: 0.8423 - loss: 0.3643 - val_binary_accuracy: 0.7743 - val_loss: 0.4730
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - binary_accuracy: 0.9115 - loss: 0.2372 - val_binary_accuracy: 0.7690 - val_loss: 0.5714
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - binary_accuracy: 0.9457 - loss: 0.1592 - val_binary_accuracy: 0.7730 - val_loss: 0.5898
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - binary_accuracy: 0.9577 - loss: 0.1268 - val_binary_accuracy: 0.7717 - val_loss: 0.6975


In [51]:
# Make some prediction probabilities to predict classes
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step


((762, 1),
 array([[0.88546306],
        [0.97468853],
        [0.9925143 ],
        [0.03310084],
        [0.00910236],
        [0.98511064],
        [0.9704325 ],
        [0.99465334],
        [0.99525875],
        [0.03101122]], dtype=float32))

In [52]:
model_3_pred = tf.squeeze(tf.round(model_3_pred_probs))
model_3_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [53]:
# Calculated model_3_results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_pred)
model_3_results

{'accuracy': 77.16535433070865,
 'precision': 0.771892513574384,
 'recall': 0.7716535433070866,
 'f1': 0.771751722782225}

### *Bidirectional RNN model*

In [59]:
# Set a random seed
tf.random.set_seed(16)

model_4_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_4")

# Build a Bidirectional RNN
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = model_4_embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1,  activation=activations.sigmoid)(x)
model_4 = keras.Model(inputs, outputs, name="model_4_bidirectional")



/home/lovecrush/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [61]:
model_4.compile(loss=losses.binary_crossentropy,
                optimizer=optimizers.Adam(),
                metrics=[metrics.binary_accuracy])

model_4.summary()

Model: "model_4_bidirectional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_4 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 15, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,477,761 (5.64 MB)

 Trainable params: 1,477,761 (5.64 MB)

 Non-trainable params: 0 (0.00 B)

In [63]:
model_4_history = model_4.fit(train_sentences, train_labels,
                              validation_data=(val_sentences, val_labels),
                              epochs=5,
                              callbacks=[create_tensorboard_callback("logs/model_4_training/","Bidirectional_RNN")])

Saving TensorBoard log files to: logs/model_4_training//Bidirectional_RNN/20250209-202133
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - binary_accuracy: 0.6912 - loss: 0.5815 - val_binary_accuracy: 0.7927 - val_loss: 0.4590
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - binary_accuracy: 0.8601 - loss: 0.3444 - val_binary_accuracy: 0.7743 - val_loss: 0.4751
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - binary_accuracy: 0.9277 - loss: 0.2116 - val_binary_accuracy: 0.7743 - val_loss: 0.5284
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - binary_accuracy: 0.9567 - loss: 0.1333 - val_binary_accuracy: 0.7651 - val_loss: 0.6732
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 15s 68ms/step - binary_accuracy: 0.9604 - loss: 0.1153 - val_binary_accuracy: 0.7638 - val_loss: 0.7853


In [65]:
# make the prediction with bidirectional RNN on validation data
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs

model_4_pred = tf.squeeze(tf.round(model_4_pred_probs))
model_4_pred

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step


<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
       1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1.,
       0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [69]:
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_pred)
model_4_results

{'accuracy': 76.37795275590551,
 'precision': 0.7638949621588562,
 'recall': 0.7637795275590551,
 'f1': 0.7638319711031701}

### *Convolutional Neural Net for Text*

In [70]:
# Test out the embedding, 1D convolutional and max pooling
embedding_test = embedding(text_vectorizer(["This is a test sentences"]))           # Turn target sentence into embedding
conv_1d = layers.Conv1D(filters=32, kernel_size=5, activation=activations.relu)     # convolve over target sequence 5 words at a time
conv_1d_output = conv_1d(embedding_test)                                            # pass embedding through 1D convolutional layer
max_pool = layers.GlobalAveragePooling1D()
max_pool_output = max_pool(conv_1d_output)                                          # Get the most important feature
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [72]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.04428809, 0.02009461, 0.02015783, 0.01728459, 0.01436211,
        0.03282959, 0.01282405, 0.01417735, 0.0078524 , 0.04640746,
        0.01236401, 0.01007239, 0.00430624, 0.01215202, 0.04858096,
        0.03593762, 0.00357465, 0.01310107, 0.01803437, 0.00566049,
        0.00415561, 0.0517248 , 0.02469531, 0.00773903, 0.0123663 ,
        0.03617433, 0.02801114, 0.01149212, 0.00571532, 0.00765486,
        0.02560461, 0.02381101]], dtype=float32)>

In [87]:
model_5 = keras.Sequential([
    layers.Input(shape=(1,), dtype=tf.string),
    layers.TextVectorization(max_tokens=max_vocab_length,
                                            output_mode="int",
                                            output_sequence_length=max_length),
    layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_5"),
    layers.Conv1D(32, 5, activation=activations.relu),
    layers.GlobalMaxPool1D(),
    layers.Dense(1, activation=activations.sigmoid)

], name="model_5_sequentialAPI")

model_5.compile(loss=losses.binary_crossentropy,
                optimizer=optimizers.Adam(),
                metrics=[metrics.binary_accuracy])

model_5.summary()

/home/lovecrush/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "model_5_sequentialAPI"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization_3            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_5 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 11, 32)         │        20,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_2          │ (None, 32)             │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,300,545 (4.96 MB)

 Trainable params: 1,300,545 (4.96 MB)

 Non-trainable params: 0 (0.00 B)

In [74]:
# Set random seed and create embedding layer (new embedding layers for each model)
tf.random.set_seed(16)

model_5_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_5")

# Create 1-dimensional convolutional layer to model sequence
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = model_5_embedding(x)
x = layers.Conv1D(32, 5, activation=activations.relu)(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation=activations.sigmoid)(x)
model_5 = keras.Model(inputs, outputs, name="model_5_Conv1D")

# Compile model
model_5.compile(loss=losses.binary_crossentropy,
                optimizer=optimizers.Adam(),
                metrics=[metrics.binary_accuracy])

model_5.summary()


/home/lovecrush/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "model_5_Conv1D"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_5 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 11, 32)         │        20,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 32)             │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,300,545 (4.96 MB)

 Trainable params: 1,300,545 (4.96 MB)

 Non-trainable params: 0 (0.00 B)

In [76]:
model_5_history = model_5.fit(train_sentences, train_labels,
                              validation_data=(val_sentences, val_labels),
                              epochs=5,
                              callbacks=[create_tensorboard_callback("logs/model_5_training/","Conv1D")])

Saving TensorBoard log files to: logs/model_5_training//Conv1D/20250209-211739
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - binary_accuracy: 0.9740 - loss: 0.0773 - val_binary_accuracy: 0.7598 - val_loss: 0.7209
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - binary_accuracy: 0.9762 - loss: 0.0665 - val_binary_accuracy: 0.7559 - val_loss: 0.7544
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - binary_accuracy: 0.9776 - loss: 0.0601 - val_binary_accuracy: 0.7585 - val_loss: 0.7634
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - binary_accuracy: 0.9773 - loss: 0.0553 - val_binary_accuracy: 0.7625 - val_loss: 0.7699
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - binary_accuracy: 0.9791 - loss: 0.0514 - val_binary_accuracy: 0.7638 - val_loss: 0.7794


In [78]:
model_5_pred = tf.squeeze(tf.round(model_5.predict(val_sentences)))
model_5_results = calculate_results(y_pred=model_5_pred, y_true=val_labels)
model_5_results

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


{'accuracy': 76.37795275590551,
 'precision': 0.7633913718531662,
 'recall': 0.7637795275590551,
 'f1': 0.7633947072405423}

### *Pretrained Embedding - Transfer learning for NLP*

### *Save model*

In [82]:
model_5.save("model_5_nlp.h5")

### *Check model on Test set*

In [85]:
# Making predictions on the test dataset
test_sentences = test_df["text"].to_list()
test_samples = random.sample(test_sentences, 10)
for test_sample in test_samples:
  pred_prob = tf.squeeze(model_5.predict([test_sample])) # has to be list
  pred = tf.round(pred_prob)
  print(f"Pred: {int(pred)}, Prob: {pred_prob}")
  print(f"Text:\n{test_sample}\n")
  print("----\n")

ValueError: Unrecognized data type: x=['Texas Seeks Comment on Rules for Changes to Windstorm Insurer: The Texas Department of Insurance is seeking pu... http://t.co/NGF9n8Jquu'] (of type <class 'list'>)